**Heart Disease Dataset**  
Este conjunto de datos se utiliza para predecir la presencia de enfermedades cardíacas en pacientes, basado en diversas características médicas. Incluye 303 muestras y 14 características. Puedes encontrarlo en el [Repositorio de Machine Learning de la UCI](https://archive.ics.uci.edu/ml/datasets/Heart+Disease).

In [3]:
!pip install ucimlrepo

### 1. **Carga de Datos**

Primero, descarga el dataset desde el [Repositorio de Machine Learning de la UCI](https://archive.ics.uci.edu/ml/datasets/Heart+Disease) y cárgalo en un DataFrame de `pandas`:

In [12]:
import pandas as pd

# Cargar el dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
column_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
df = pd.read_csv(url, names=column_names)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


### 2. **Preprocesamiento de Datos**

**Manejo de valores faltantes**: Verificar y manejar los valores faltantes en el dataset.


In [14]:
# Reemplazar los signos de interrogación con NaN y eliminar filas con valores faltantes
df.replace('?', pd.NA, inplace=True)
df.dropna(inplace=True)

**Conversión de tipos de datos**: Asegúrate de que todas las columnas tengan el tipo de dato adecuado.


In [16]:
# Convertir columnas a tipo numérico
df = df.apply(pd.to_numeric)

**Codificación de variables categóricas**: Si es necesario, convierte las variables categóricas en variables dummy.


In [17]:
df = pd.get_dummies(df, columns=['cp', 'restecg', 'slope', 'thal'], drop_first=True)


- **Revisión del objetivo**: La columna `target` indica la presencia de enfermedad cardíaca. Convierte los valores mayores a 0 en 1 para una clasificación binaria.


In [41]:
# Convertir el objetivo en binario
df['target'] = df['target'].apply(lambda x: -1 if x == 0 else 1)

### 3. **División de Datos**

Divide el dataset en conjuntos de entrenamiento y prueba.

In [42]:
from sklearn.model_selection import train_test_split

# Separar características y objetivo
X = df.drop('target', axis=1)
y = df['target']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4. **Normalización de Datos**

Normaliza las características para mejorar el rendimiento del modelo.

In [43]:
from sklearn.preprocessing import StandardScaler

# Inicializar el escalador
scaler = StandardScaler()

# Ajustar y transformar los datos de entrenamiento, y transformar los datos de prueba
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
import os
pwd = "/content/drive/MyDrive/Colab Notebooks/micrograd/"
os.chdir(pwd)
!pwd

/content/drive/MyDrive/Colab Notebooks/micrograd


### 5. **Inicialización del Modelo**




Primero, crea una instancia de tu MLP especificando el número de entradas y la arquitectura de las capas ocultas. Por ejemplo, si tienes 13 características de entrada y deseas dos capas ocultas con 16 neuronas cada capa:

In [48]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from micrograd.engine import Value
from micrograd.nn import Neuron, Layer, MLP

# Inicializar el modelo con 13 entradas y dos capas ocultas
model = MLP(13, [16, 16, 1])
np.random.seed(1337)
random.seed(1337)

### 6. **Definición de la Función de Pérdida**


Implementa una función de pérdida adecuada para tu tarea de clasificación binaria. Una opción común es la pérdida de margen máximo (SVM)

In [49]:
# loss function
def loss(batch_size=None):

    # inline DataLoader :)
    if batch_size is None:
        Xb, yb = X_train, y_train
    else:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb, yb = X_train[ri], y_train[ri]
    inputs = [list(map(Value, xrow)) for xrow in Xb]

    # forward the model to get scores
    scores = list(map(model, inputs))

    # svm "max-margin" loss
    losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = data_loss + reg_loss

    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

total_loss, acc = loss()
print(total_loss, acc)

Value(data=0.49402945016507976, grad=0) 0.7763713080168776


### 7. **Ciclo de Entrenamiento**

Implementa el ciclo de entrenamiento que incluye la propagación hacia adelante, el cálculo de la pérdida, la retropropagación y la actualización de los parámetros:

In [50]:
# optimization
for k in range(100):

    # forward
    total_loss, acc = loss()

    # backward
    model.zero_grad()
    total_loss.backward()

    # update (sgd)
    # learning_rate = 1.0 - 0.9*k/100
    learning_rate = 0.01
    for p in model.parameters():
        p.data -= learning_rate * p.grad

    if k % 1 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")

step 0 loss 0.49402945016507976, accuracy 77.63713080168776%
step 1 loss 0.4486180955760732, accuracy 79.74683544303798%
step 2 loss 0.4115871750528911, accuracy 82.70042194092827%
step 3 loss 0.38024613774146754, accuracy 86.49789029535864%
step 4 loss 0.35273209211741074, accuracy 87.76371308016878%
step 5 loss 0.3274010896231087, accuracy 88.60759493670885%
step 6 loss 0.30384201833480134, accuracy 89.0295358649789%
step 7 loss 0.2846352523516884, accuracy 89.87341772151899%
step 8 loss 0.2662616635597411, accuracy 90.71729957805907%
step 9 loss 0.24946004394671828, accuracy 91.13924050632912%
step 10 loss 0.23277718759947366, accuracy 91.9831223628692%
step 11 loss 0.2165378340652727, accuracy 93.24894514767934%
step 12 loss 0.2018548922814956, accuracy 94.51476793248945%
step 13 loss 0.18801195263549655, accuracy 94.9367088607595%
step 14 loss 0.1748577212030588, accuracy 95.35864978902954%
step 15 loss 0.16318941290446065, accuracy 96.20253164556962%
step 16 loss 0.15323480733703

## 8. **Evaluación del Modelo**

Después del entrenamiento, evalúa el rendimiento del modelo en un conjunto de datos de prueba para verificar su capacidad de generalización:



In [51]:
correct_predictions = 0

for x, y in zip(X_test, y_test):
    inputs = [Value(xi) for xi in x]
    output = model(inputs)
    predicted_label = 1 if output.data > 0 else -1
    if predicted_label == y:
        correct_predictions += 1

accuracy = correct_predictions / len(X_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

Precisión en el conjunto de prueba: 0.9833
